## Data Preparation

### Data Preporocessing


In [6]:
# Loading the packages I will need:

print("Loading packages...", end = '')

import numpy as np
from pyunpack import Archive 
import pandas as pd 
import missingno as msno
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error 

import zipfile
import shutil  # For handling directories


print("Successful.")

Loading packages...Successful.


In [7]:
# Paths
main_dir = "data"  # Main directory containing year folders
output_dir = "./temp_extracted_files"  # Temporary directory for extracted files

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# List to store dataframes
dataframes = []

# Iterate over year folders
for year in os.listdir(main_dir):
    year_path = os.path.join(main_dir, year)
    if os.path.isdir(year_path):  # Ensure it's a folder
        print(f"Processing year folder: {year}")
        
        # Process each .zip file within the year folder
        for month_zip in os.listdir(year_path):
            month_zip_path = os.path.join(year_path, month_zip)
            
            if month_zip.endswith(".zip"):
                print(f"Extracting: {month_zip}")
                # Extract the .zip file
                with zipfile.ZipFile(month_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(output_dir)
                
                # Process the extracted files
                extracted_files = os.listdir(output_dir)
                print(f"Extracted files: {extracted_files}")
                
                for extracted_file in extracted_files:
                    extracted_file_path = os.path.join(output_dir, extracted_file)
                    
                    # Check for .csv and .xlsx files
                    if extracted_file.endswith(".csv"):
                        print(f"Reading CSV file: {extracted_file}")
                        df = pd.read_csv(extracted_file_path)
                        df['Year'] = year  # Add a column for the year
                        dataframes.append(df)
                    
                    elif extracted_file.endswith((".xlsx", ".xls")):
                        print(f"Reading Excel file: {extracted_file}")
                        df = pd.read_excel(extracted_file_path)
                        df['Year'] = year  # Add a column for the year
                        dataframes.append(df)
                
                # Clean up the temporary directory
                for temp_file in extracted_files:
                    temp_file_path = os.path.join(output_dir, temp_file)
                    if os.path.isfile(temp_file_path):
                        os.remove(temp_file_path)  # Remove files
                    elif os.path.isdir(temp_file_path):
                        shutil.rmtree(temp_file_path)  # Remove directories

# Combine dataframes
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_file = "combined_data.csv"
    combined_df.to_csv(combined_file, index=False)
    print(f"Combined data saved as {combined_file}")
else:
    print("No valid files found to combine.")




Processing year folder: 2020
Extracting: April2020TransBorderRawData.zip
Extracted files: ['Apr 2020']
Extracting: August2020TransBorderRawData.zip
Extracted files: ['August2020TransBorderRawData']
Extracting: February2020TransBorderRawData.zip
Extracted files: ['Feb 2020']
Extracting: January2020TransBorderRawData.zip
Extracted files: ['Jan 2020']
Extracting: July2020TransBorderRawData.zip
Extracted files: ['dot1_0720.csv', 'dot1_ytd_0720.csv', 'dot2_0720.csv', 'dot2_ytd_0720.csv', 'dot3_0720.csv', 'dot3_ytd_0720.csv', '__MACOSX']
Reading CSV file: dot1_0720.csv
Reading CSV file: dot1_ytd_0720.csv
Reading CSV file: dot2_0720.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\1280360659.py:37: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extracted_file_path)


Reading CSV file: dot2_ytd_0720.csv
Reading CSV file: dot3_0720.csv
Reading CSV file: dot3_ytd_0720.csv
Extracting: June2020TransBorderRawData.zip
Extracted files: ['dot1_0620.csv', 'dot1_ytd_0620.csv', 'dot2_0620.csv', 'dot2_ytd_0620.csv', 'dot3_0620.csv', 'dot3_ytd_0620.csv']
Reading CSV file: dot1_0620.csv
Reading CSV file: dot1_ytd_0620.csv
Reading CSV file: dot2_0620.csv
Reading CSV file: dot2_ytd_0620.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\1280360659.py:37: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extracted_file_path)


Reading CSV file: dot3_0620.csv
Reading CSV file: dot3_ytd_0620.csv
Extracting: March2020TransBorderRawData.zip
Extracted files: ['Mar 2020']
Extracting: May2020TransBorderRawData.zip
Extracted files: ['May 2020']
Extracting: September2020TransBorderRawData.zip
Extracted files: ['September2020TransBorderRawData']
Processing year folder: 2021
Extracting: April2021TransBorderRawData.zip
Extracted files: ['April2021TransBorderRawData']
Extracting: February2021TransBorderRawData.zip
Extracted files: ['February2021TransBorderRawData']
Extracting: January2021TransBorderRawData.zip
Extracted files: ['January2021TransBorderRawData']
Extracting: July-to-Dec-2021.zip
Extracted files: ['New folder']
Extracting: July2021TransBorderRawData.zip
Extracted files: ['July2021TransBorderRawData']
Extracting: June2021TransBorderRawData.zip
Extracted files: ['June2021TransBorderRawData']
Extracting: March2021TransBorderRawData.zip
Extracted files: ['March2021TransBorderRawData']
Extracting: May2021TransBor

...

Reading CSV file: dot3_1223.csv
Extracting: Feb2023.zip
Extracted files: ['Feb2023']
Extracting: Jan2023.zip
Extracted files: ['Jan2023']
Extracting: July2023.zip
Extracted files: ['July2023']
Extracting: June2023.zip
Extracted files: ['June2023']
Extracting: March2023.zip
Extracted files: ['March2023']
Extracting: May2023.zip
Extracted files: ['May2023']
Extracting: Nov2023.zip
Extracted files: ['Nov2023']
Extracting: Oct2023.zip
Extracted files: ['Oct2023']
Extracting: sept2023.zip
Extracted files: ['sept2023']
Processing year folder: 2024
Extracting: April2024.zip
Extracted files: ['April2024']
Extracting: August2024.zip
Extracted files: ['dot1_0824.csv', 'dot2_0824.csv', 'dot3_0824.csv']
Reading CSV file: dot1_0824.csv
Reading CSV file: dot2_0824.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\1280360659.py:37: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extracted_file_path)


Reading CSV file: dot3_0824.csv
Extracting: Feb2024.zip
Extracted files: ['Feb2024']
Extracting: Jan2024.zip
Extracted files: ['Jan2024']
Extracting: July2024.zip
Extracted files: ['July2024']
Extracting: June2024.zip
Extracted files: ['June2024']
Extracting: March2024.zip
Extracted files: ['March2024']
Extracting: May2024.zip
Extracted files: ['May2024']
Extracting: september2024.zip
Extracted files: ['dot1_0924.csv', 'dot2_0924.csv', 'dot3_0924.csv']
Reading CSV file: dot1_0924.csv
Reading CSV file: dot2_0924.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\1280360659.py:37: DtypeWarning: Columns (4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(extracted_file_path)


Reading CSV file: dot3_0924.csv
Combined data saved as combined_data.csv


In [8]:
# Read the CSV file from the root directory
combined_df = pd.read_csv("combined_data.csv")

# Display the first five rows to inspect
print(combined_df.head())


C:\Users\DELL\AppData\Local\Temp\ipykernel_15088\1203341999.py:2: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv("combined_data.csv")


   TRDTYPE USASTATE  DEPE  DISAGMOT MEXSTATE CANPROV  COUNTRY  VALUE  SHIPWT  \
0        1       AK  0712         5      NaN      XQ     1220  12182       0   
1        1       AK  20XX         3      NaN      XA     1220  29921    1209   
2        1       AK  20XX         3      NaN      XA     1220   2590      16   
3        1       AK  20XX         3      NaN      XC     1220  58967    7843   
4        1       AK  20XX         3      NaN      XC     1220   7201       1   

   FREIGHT_CHARGES   DF CONTCODE  MONTH  YEAR  Year  COMMODITY2  
0              461  1.0        X      7  2020  2020         NaN  
1              202  1.0        X      7  2020  2020         NaN  
2               74  2.0        X      7  2020  2020         NaN  
3              857  1.0        X      7  2020  2020         NaN  
4              133  2.0        X      7  2020  2020         NaN  


In [9]:
# Check the columns and their data types
combined_df.dtypes


TRDTYPE              int64
USASTATE            object
DEPE                object
DISAGMOT             int64
MEXSTATE            object
CANPROV             object
COUNTRY              int64
VALUE                int64
SHIPWT               int64
FREIGHT_CHARGES      int64
DF                 float64
CONTCODE            object
MONTH                int64
YEAR                 int64
Year                 int64
COMMODITY2         float64
dtype: object

In [10]:
# Display the last five rows of the DataFrame
combined_df.tail()


,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,Year,COMMODITY2
2051246,2,NaN,55XX,5,NaN,NaN,1220,4384342,7399,1883,NaN,0,9,2024,2024,98.0
2051247,2,NaN,55XX,8,NaN,NaN,1220,50211,6350,3500,NaN,0,9,2024,2024,98.0
2051248,2,NaN,60XX,8,NaN,NaN,1220,793390,80,500,NaN,0,9,2024,2024,89.0
2051249,2,NaN,70XX,8,NaN,NaN,1220,233990301,0,0,NaN,0,9,2024,2024,99.0
2051250,2,NaN,70XX,8,NaN,NaN,2010,224981722,0,0,NaN,0,9,2024,2024,99.0


In [11]:
# check first five rows

combined_df.head()

,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,Year,COMMODITY2
0,1,AK,0712,5,NaN,XQ,1220,12182,0,461,1.0,X,7,2020,2020,NaN
1,1,AK,20XX,3,NaN,XA,1220,29921,1209,202,1.0,X,7,2020,2020,NaN
2,1,AK,20XX,3,NaN,XA,1220,2590,16,74,2.0,X,7,2020,2020,NaN
3,1,AK,20XX,3,NaN,XC,1220,58967,7843,857,1.0,X,7,2020,2020,NaN
4,1,AK,20XX,3,NaN,XC,1220,7201,1,133,2.0,X,7,2020,2020,NaN


In [12]:
# Check the total number of rows and columns in the DataFrame
combined_df.shape


(2051251, 16)

In [16]:
# Compare the 'YEAR' and 'Year' columns to check for any differences in shape
combined_df['YEAR'].shape



(2051251,)

In [17]:
combined_df['Year'].shape

(2051251,)

In [21]:
# Drop the 'Year' column since the two columns are literally the same 
# combined_df = combined_df.drop(columns=["Year"])

# Check the updated DataFrame
combined_df.head(10)


,TRDTYPE,USASTATE,DEPE,DISAGMOT,MEXSTATE,CANPROV,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,CONTCODE,MONTH,YEAR,COMMODITY2
0,1,AK,0712,5,NaN,XQ,1220,12182,0,461,1.0,X,7,2020,NaN
1,1,AK,20XX,3,NaN,XA,1220,29921,1209,202,1.0,X,7,2020,NaN
2,1,AK,20XX,3,NaN,XA,1220,2590,16,74,2.0,X,7,2020,NaN
3,1,AK,20XX,3,NaN,XC,1220,58967,7843,857,1.0,X,7,2020,NaN
4,1,AK,20XX,3,NaN,XC,1220,7201,1,133,2.0,X,7,2020,NaN
5,1,AK,20XX,3,NaN,XO,1220,76586,846,819,1.0,X,7,2020,NaN
6,1,AK,20XX,3,NaN,XO,1220,6967,18,136,2.0,X,7,2020,NaN
7,1,AK,20XX,3,NaN,XQ,1220,35010,2934,2482,1.0,X,7,2020,NaN
8,1,AK,2301,5,TB,NaN,2010,43200,0,0,1.0,0,7,2020,NaN
9,1,AK,2303,5,CO,NaN,2010,7065,0,0,1.0,0,7,2020,NaN


In [24]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051251 entries, 0 to 2051250
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   TRDTYPE          int64  
 1   USASTATE         object 
 2   DEPE             object 
 3   DISAGMOT         int64  
 4   MEXSTATE         object 
 5   CANPROV          object 
 6   COUNTRY          int64  
 7   VALUE            int64  
 8   SHIPWT           int64  
 9   FREIGHT_CHARGES  int64  
 10  DF               float64
 11  CONTCODE         object 
 12  MONTH            int64  
 13  YEAR             int64  
 14  COMMODITY2       float64
dtypes: float64(2), int64(8), object(5)
memory usage: 234.7+ MB


In [25]:
# Check for null values in each column
combined_df.isnull().sum()


TRDTYPE                  0
USASTATE            297709
DEPE               1283008
DISAGMOT                 0
MEXSTATE           1485680
CANPROV             977810
COUNTRY                  0
VALUE                    0
SHIPWT                   0
FREIGHT_CHARGES          0
DF                  693632
CONTCODE                 0
MONTH                    0
YEAR                     0
COMMODITY2          470534
dtype: int64

#### change fo some dtypes
COMMODITY2 from float to int

Container Code (CONTCODE) Code Containerized? 
X Containerized 
0 Non-Containerized 
 
  DF from float to int

  Country Code (COUNTRY) 
Code Country 
1220 Canada 
2010 Mexico 
 
Domestic/Foreign Code (DF)   float to object 
Distinguishes whether the merchandise was produced in the U.S. 
Code 
1 domestically produced merchandise 
2 foreign produced merchandise  


Mode of Transportation Code (DISAGMOT)  int to object 
Code Description 
1 Vessel 
3 Air 
4 Mail (U.S. Postal Service) 
5 Truck 
6 Rail 
7 Pipeline 
8 Other 
9 Foreign Trade Zones (FTZs) 


Month (MONTH)    int to object
Month 
Code Month 
  1 January 
  2 February 
  3 March 
  4 April 
  5 May 
  6 June 
  7 July 
  8 August 
  9 September 
10 October 
11 November 
12 December



Trade Type Code (TRDTYPE)     int to obj
Code Trade Type 
1 Export 
2 Import  vvvvvvvvv



Value (VALUE) 
Value of Goods in United States Dollars 
 
Year (YEAR)  int to datetime


Freight Charges (FREIGHT_CHARGES)   no need
Freight Charges in U.S. Dollars


MONTH  int to date time 

In [26]:
combined_df.describe()

,TRDTYPE,DISAGMOT,COUNTRY,VALUE,SHIPWT,FREIGHT_CHARGES,DF,MONTH,YEAR,COMMODITY2
count,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,2.051251e+06,1.357619e+06,2.051251e+06,2.051251e+06,1.580717e+06
mean,1.338151e+00,4.756261e+00,1.528714e+03,2.390849e+06,1.209763e+06,3.337063e+04,1.325863e+00,5.091202e+00,2.020663e+03,5.652508e+01
std,4.730802e-01,1.251327e+00,3.854605e+02,3.027425e+07,3.780300e+07,8.343282e+05,4.686966e-01,2.895490e+00,1.426718e+00,2.792732e+01
min,1.000000e+00,1.000000e+00,1.220000e+03,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.020000e+03,1.000000e+00
25%,1.000000e+00,5.000000e+00,1.220000e+03,1.355500e+04,0.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,2.020000e+03,3.300000e+01
50%,1.000000e+00,5.000000e+00,1.220000e+03,6.490400e+04,0.000000e+00,2.840000e+02,1.000000e+00,5.000000e+00,2.020000e+03,5.900000e+01
75%,2.000000e+00,5.000000e+00,2.010000e+03,3.778740e+05,2.624000e+03,2.847000e+03,2.000000e+00,7.000000e+00,2.020000e+03,8.400000e+01
max,2.000000e+00,9.000000e+00,2.010000e+03,5.595625e+09,8.110661e+09,2.046194e+08,2.000000e+00,1.200000e+01,2.024000e+03,9.900000e+01


In [ ]:
# Change the dtype of TRDTYPE from int to string and replace values
combined_df['TRDTYPE'] = combined_df['TRDTYPE'].astype(str).replace({'1': 'Export', '2': 'Import'})

# Change COMMODITY2 from float to int
combined_df['COMMODITY2'] = combined_df['COMMODITY2'].astype(str)

# Replace values in CONTCODE
combined_df['CONTCODE'] = combined_df['CONTCODE'].replace({'X': 'Containerized', '0': 'Non-Containerized'})

# Change DF from float to int and then to string, replace values
combined_df['DF'] = combined_df['DF'].astype(int).astype(str).replace({'1': 'domestic merchandise', '2': 'foreign merchandise'})

# Replace values in COUNTRY
combined_df['COUNTRY'] = combined_df['COUNTRY'].replace({1220: 'Canada', 2010: 'Mexico'})

# Change DISAGMOT from int to string and replace values
combined_df['DISAGMOT'] = combined_df['DISAGMOT'].astype(str).replace({
    '1': 'Vessel', '3': 'Air', '4': 'Mail (U.S. Postal Service)', '5': 'Truck',
    '6': 'Rail', '7': 'Pipeline', '8': 'Other', '9': 'Foreign Trade Zones (FTZs)'
})

# Change MONTH from int to string and replace values
combined_df['MONTH'] = combined_df['MONTH'].astype(str).replace({
    '1': 'January', '2': 'February', '3': 'March', '4': 'April', '5': 'May', '6': 'June',
    '7': 'July', '8': 'August', '9': 'September', '10': 'October', '11': 'November', '12': 'December'
})

# Change YEAR from int to datetime
combined_df['YEAR'] = pd.to_datetime(combined_df['YEAR'], format='%Y')

# Verify the changes
combined_df.info()